<a href="https://colab.research.google.com/github/aiuser53/test/blob/main/MDX_Net_Colab_(TR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDX-Net Colab
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://github.githubassets.com/pinned-octocat.svg" alt="icon1" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>Bu not defterinde verilen eğitimli modeller <a href="https://github.com/Anjok07">UVR-GUI</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://github.com/Anjok07/ultimatevocalremovergui/raw/master/gui_data/img/GUI-Icon.ico" alt="icon2" style="margin-right:10px; height: 20px;margin-top:10px" width="1.5%">
  <span>RESMİ UVR GITHUB SAYFASI: <a href="https://github.com/Anjok07/ultimatevocalremovergui">here</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://avatars.githubusercontent.com/u/24620594" alt="icon3" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>RESMİ CLI Versiyonu: <a href="https://github.com/tsurumeso/vocal-remover">here</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://icons.getbootstrap.com/assets/icons/discord.svg" alt="icon4" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>Bize katıl <a href="https://cutt.ly/0TcDjmo">Discord sunucumuz</a>!</span>
</div>
<sup><br>Ultimate Vocal Remover (unofficial)</sup>
<sup><br>MDX-Net <a href="https://github.com/kuielab">kuielab</a> tarafından ve <a href="https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw">AudioHacker</a> Colaboratory'ye uyarlanmıştır

<sup><br>Desteğiniz benim için çok önemli. Çalışmalarımı beğeniyorsanız, ko-fi almayı unutmayın:<br></sup>
[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/X8X6M8FR0)

In [ ]:
import json
import os
import os.path
import gc
import psutil
import requests
import subprocess
import glob
import time
import logging
import sys
from bs4 import BeautifulSoup
from google.colab import drive, files, output
from IPython.display import Audio, display

if "first_cell_ran" in locals():
    print("Bu hücreyi bu oturum için çalıştırdınız. Bir şeylerin yanlış gittiğini düşünüyorsanız veya montaj yolunu değiştirmek istiyorsanız, çalışma zamanını yeniden başlatın.")
else:
    print('Kuruluyor... lütfen 1-2 dakika kadar bekleyin.')

    branch = "https://github.com/NaJeongMo/Colab-for-MDX_B"

    model_params = "https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/model_data.json"
    _Models = "https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/"
    # _models = "https://pastebin.com/raw/jBzYB8vz"
    _models = "https://raw.githubusercontent.com/TRvlvr/application_data/main/filelists/download_checks.json"
    stem_naming = "https://pastebin.com/raw/mpH4hRcF"
    arl_check_endpoint = 'https://dz.doubledouble.top/check' # param: arl?=<>

    file_folder = "Colab-for-MDX_B"

    model_ids = requests.get(_models).json()
    model_ids = model_ids["mdx_download_list"].values()

    model_params = requests.get(model_params).json()
    stem_naming = requests.get(stem_naming).json()

    os.makedirs("tmp_models", exist_ok=True)

    # @markdown Google Drive'ı bağlamak istemiyorsanız, bu kutunun işaretini kaldırın.
Drive_bagla = False #@param {type:"boolean"}
    # @markdown Bağlanacak sürücü için yol: Düzgün yapılmazsa sorunlara neden olabileceğinden lütfen bunu değiştirirken dikkatli olun.
    mounting_path = "/content/drive/MyDrive"  # @param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
    # @markdown Güncellemeye zorla ve yerel değişiklikleri göz ardı et: deponuzdaki tüm yerel değişiklikleri atar, tüm dosyaları orijinal commit'teki sürümlerle etkili bir şekilde değiştirir.
guncellemeyi_koru = False #@param {type:"boolean"}
    # @markdown Otomatik Güncelleme (değişikliklerinizi atmaz)
oto_guncelleme = True #@param {type:"boolean"}


    reqs_apt = []  # !sudo apt-get install
    reqs_pip = ["librosa>=0.6.3,<0.9", "onnxruntime_gpu", "deemix", "yt_dlp"]  # pip3 install

    class hide_opt:  # çıktıları gizle
        def __enter__(self):
            self._original_stdout = sys.stdout
            sys.stdout = open(os.devnull, "w")

        def __exit__(self, exc_type, exc_val, exc_tb):
            sys.stdout.close()
            sys.stdout = self._original_stdout

    def get_size(bytes, suffix="B"):  # ram'ı oku
        global svmem
        factor = 1024
        for unit in ["", "K", "M", "G", "T", "P"]:
            if bytes < factor:
                return f"{bytes:.2f}{unit}{suffix}"
            bytes /= factor
        svmem = psutil.virtual_memory()


    print('kurulum gereksinimleri...',end=' ')
    with hide_opt():
        for x in reqs_apt:
            subprocess.run(["sudo", "apt-get", "install", x])
        for x in reqs_pip:
            subprocess.run(["python3", "-m", "pip", "install", x])
    print('Bitti')

    def install_or_mount_drive():
        print(
            "Lütfen açılır sekmedeki talimatları izleyerek hesabınızda oturum açın.\nDosyaları Google Drive'ınıza yüklemek için bu adım gereklidir.\nBu not defterinin güvenliği konusunda herhangi bir endişeniz varsa, \"MountDrive\" onay kutusunun işaretini kaldırarak sürücünüzü bağlamamayı seçebilirsiniz."
        )
        drive.mount("/content/drive", force_remount=True)
        os.chdir(mounting_path)
        # önceki kurulumun yapılıp yapılmadığını kontrol edin
        if os.path.exists(os.path.join(mounting_path, file_folder)):
            # güncelleme kontrolü
            os.chdir(file_folder)

            if force_update:
                print('Güncellemeye zorlanıyor...')

                commands = [
                    ["git", "pull"],
                    ["git", "checkout", "--", "."],
                ]

                for cmd in commands:
                    subprocess.run(cmd)

            elif auto_update:
                print('Güncellemeler kontrol ediliyor...')
                commands = [
                    ["git", "pull"],
                ]

                for cmd in commands:
                    subprocess.run(cmd)
        else:
            subprocess.run(["git", "clone", "https://github.com/NaJeongMo/Colab-for-MDX_B.git"])
            os.chdir(file_folder)

    def use_uvr_without_saving():
        global mounting_path
        print("Uyarı: dosyalar kişisel sürücüye kaydedilmeyecektir.")
        print(f"Yükleniyor {file_folder}...", end=" ")
        mounting_path = "/content"
        with hide_opt():
            os.chdir(mounting_path)
            subprocess.run(["git", "clone", "https://github.com/NaJeongMo/Colab-for-MDX_B.git"])
            os.chdir(file_folder)

    if MountDrive:
        install_or_mount_drive()
    else:
        use_uvr_without_saving()
    print("Bitti")
    if not os.path.exists("tracks"):
        os.mkdir("tracks")

    print('Gerekli kütüphaneler içe aktarılıyor...',end=' ')

    import os
    import mdx
    import librosa
    import torch
    import soundfile as sf
    import numpy as np
    import yt_dlp

    from deezer import Deezer
    from deezer import TrackFormats
    import deemix
    from deemix.settings import load as loadSettings
    from deemix.downloader import Downloader
    from deemix import generateDownloadObject

    logger = logging.getLogger("yt_dlp")
    logger.setLevel(logging.ERROR)

    def id_to_ptm(mkey):
        if mkey in model_ids:
            mpath = f"/content/tmp_models/{mkey}"
            if not os.path.exists(f'/content/tmp_models/{mkey}'):
                print('Model yükleniyor...',end=' ')
                subprocess.run(
                    ["wget", _Models+mkey, "-O", mpath]
                )
                print(f'{mpath} konumuna kaydedildi')
                # get_ipython().system(f'gdown {model_id} -O /content/tmp_models/{mkey}')
                return mpath
            else:
                return mpath
        else:
            mpath = f'models/{mkey}'
            return mpath

    def prepare_mdx(custom_param=False, dim_f=None, dim_t=None, n_fft=None, stem_name=None, compensation=None):
        device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
        if custom_param:
            assert not (dim_f is None or dim_t is None or n_fft is None or compensation is None), 'Custom parameter selected, but incomplete parameters are provided.'
            mdx_model = mdx.MDX_Model(
                device,
                dim_f = dim_f,
                dim_t = dim_t,
                n_fft = n_fft,
                stem_name=stem_name,
                compensation=compensation
            )
        else:
            model_hash = mdx.MDX.get_hash(onnx)
            if model_hash in model_params:
                mp = model_params.get(model_hash)
                mdx_model = mdx.MDX_Model(
                    device,
                    dim_f = mp["mdx_dim_f_set"],
                    dim_t = 2**mp["mdx_dim_t_set"],
                    n_fft = mp["mdx_n_fft_scale_set"],
                    stem_name=mp["primary_stem"],
                    compensation=compensation if not custom_param and compensation is not None else mp["compensate"]
                )
        return mdx_model

    def run_mdx(onnx, mdx_model,filename,diff=False,suffix=None,diff_suffix=None, denoise=False):
        mdx_sess = mdx.MDX(onnx,mdx_model)
        print(f"İşleniyor: {filename}")
        wave, sr = librosa.load(filename,mono=False, sr=44100)
        # giriş dalgasını normalleştirmek daha iyi çıktı verir
        peak = max(np.max(wave), abs(np.min(wave)))
        wave /= peak
        if denoise:
            wave_processed = -(mdx_sess.process_wave(-wave)) + (mdx_sess.process_wave(wave))
            wave_processed *= 0.5
        else:
            wave_processed = mdx_sess.process_wave(wave)
        # önceki zirveye dönüş
        wave_processed *= peak

        stem_name = mdx_model.stem_name if suffix is None else suffix # use suffix if provided
        save_path = f"{os.path.basename(os.path.splitext(filename)[0])}_{stem_name}.wav"
        save_path = os.path.join(
                'separated',
                save_path
            )
        sf.write(
            save_path,
            wave_processed.T,
            sr
        )

        print(f'tamam, kaydedildi: {save_path}')

        if diff:
            diff_stem_name = stem_naming.get(stem_name) if diff_suffix is None else diff_suffix # use suffix if provided
            stem_name = f"{stem_name}_diff" if diff_stem_name is None else diff_stem_name
            save_path = f"{os.path.basename(os.path.splitext(filename)[0])}_{stem_name}.wav"
            save_path = os.path.join(
                    'separated',
                    save_path
                )
            sf.write(
                save_path,
                (-wave_processed.T*mdx_model.compensation)+wave.T,
                sr
            )
            print(f'ters çevirme tamamlandı, kaydedildi: {save_path}')
        del mdx_sess, wave_processed, wave
        gc.collect()

    def is_valid_url(url):
        import re
        regex = re.compile(
            r'^https?://'
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'
            r'localhost|'
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
            r'(?::\d+)?'
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)
        return url is not None and regex.search(url)

    def download_deezer(link, arl, fmt='FLAC'):
        match fmt:
            case 'FLAC':
                bitrate = TrackFormats.FLAC
            case 'MP3_320':
                bitrate = TrackFormats.MP3_320
            case 'MP3_128':
                bitrate = TrackFormats.MP3_128
            case _:
                bitrate = TrackFormats.MP3_128

        dz = Deezer()
        settings = loadSettings('dz_config')
        settings['downloadLocation'] = './tracks'
        if not dz.login_via_arl(arl.strip()):
            raise Exception('Error while logging in with provided ARL.')
        downloadObject = generateDownloadObject(dz, link, bitrate)
        print(f'Yükleniyor {downloadObject.type}: "{downloadObject.title}" by {downloadObject.artist}...',end=' ',flush=True)
        Downloader(dz, downloadObject, settings).start()
        print(f'Bitti.')

        path_to_audio = []
        for file in downloadObject.files:
            path_to_audio.append(file["path"])

        return path_to_audio

    def download_link(url):
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': '%(title)s.%(ext)s',
            'nocheckcertificate': True,
            'ignoreerrors': True,
            'no_warnings': True,
            'extractaudio': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(url, download=True)
            download_path = ydl.prepare_filename(result)
        return download_path

    print('Kurulum bitti!')
    first_cell_ran = True

In [ ]:
if 'first_cell_ran' in locals():
    os.chdir(mounting_path + '/' + file_folder + '/')
    #parameter markdowns-----------------
    #@markdown ### Girdi dosyası
    #@markdown track dosya adı: Şarkılarınızı "tracks" klasörüne yükleyin. ile bölerek birden fazla bağlantı/dosya sağlayabilirsiniz;
    dosya adı = "" #@param {type:"string"}
    #@markdown onnx modeli (eğer kendi modeliniz varsa, models klasörüne yükleyin)
    onnx = "UVR-MDX-NET-Inst_HQ_3.onnx" #@param ["Kim_Inst.onnx", "Kim_Vocal_1.onnx", "Kim_Vocal_2.onnx", "kuielab_a_bass.onnx", "kuielab_a_drums.onnx", "kuielab_a_other.onnx", "kuielab_a_vocals.onnx", "kuielab_b_bass.onnx", "kuielab_b_drums.onnx", "kuielab_b_other.onnx", "kuielab_b_vocals.onnx", "Reverb_HQ_By_FoxJoy.onnx", "UVR-MDX-NET-Inst_1.onnx", "UVR-MDX-NET-Inst_2.onnx", "UVR-MDX-NET-Inst_3.onnx", "UVR-MDX-NET-Inst_HQ_1.onnx", "UVR-MDX-NET-Inst_HQ_2.onnx", "UVR-MDX-NET-Inst_Main.onnx", "UVR_MDXNET_1_9703.onnx", "UVR_MDXNET_2_9682.onnx", "UVR_MDXNET_3_9662.onnx", "UVR_MDXNET_9482.onnx", "UVR_MDXNET_KARA.onnx", "UVR_MDXNET_KARA_2.onnx", "UVR_MDXNET_Main.onnx", "UVR-MDX-NET-Inst_HQ_3.onnx"]{allow-input: true}
    #@markdown tümünü işle: bunun yerine tracks/ klasörü içindeki tüm parçaları işler. (dosya adı göz ardı edilecektir!)
hepsini_uygula = False #@param {type:"boolean"}


    #@markdown ### Ayarlar
    #@markdown invert: giriş ve çıkış arasındaki farkı elde edin (örneğin Vokallerden Enstrümantal elde edin)
ters_cevir = True #@param {type:"boolean"}
    #@markdown denoise: MDX gürültüsünden kurtulun. (Bu, giriş parçasını iki kez işler)
    denoise = True  # @param{type:"boolean"}

    #@markdown ### Özel model parametreleri (Bunu yalnızca yeni/resmi olmayan/özel modeller kullanıyorsanız kullanın)
    #@markdown Özel model parametrelerini kullan. (Varsayılan: işaretli değil veya otomatik)
ozel_parametre_kullan = False #@param {type:"boolean"}
    #@markdown Çıktı dosyası soneki (genellikle kök adı, örneğin Vokaller)
    suffix = "Vokaller_özel" #@param ["Vocaller", "Davullar", "Basslar", "Diğerr"]{allow-input: true}
    suffix_invert = "Enstrümantal_özel" #@param ["Instrumental", "Davulsuz", "Bassız", "Enstrümanlar"]{allow-input: true}
    #@markdown Model parameterleri
    dim_f = 3072 #@param {type: "integer"}
    dim_t = 256 #@param {type: "integer"}
    n_fft = 6144 #@param {type: "integer"}
    #@markdown özel telafi kullan: yalnızca modeliniz için kendi telafi değeriniz varsa. use_custom_parameter işaretli olmasa bile bu yine de geçerlidir (Varsayılan: işaretli değil veya otomatik)
ozel_compensation_kullan = False #@param {type:"boolean"}
    compensation = 1.000 #@param {type: "number"}

    #@markdown ### Ekstralar
    #@markdown Deezer arl: doğrudan deezer parçaları için ARL'nizi buraya yapıştırın!
    arl = "" #@param {type:"string"}
    #@markdown Parça formatı: parça kalitesini/formatını seçin
    parça_formatı = "FLAC" #@param ["FLAC","MP3_320","MP3_128"]
    #@markdown Çalışmada kullanılan yazdırma ayarları
    print_settings = False  # @param{type:"boolean"}



    onnx = id_to_ptm(onnx)
    compensation = compensation if use_custom_compensation or use_custom_parameter else None
    mdx_model = prepare_mdx(use_custom_parameter, dim_f, dim_t, n_fft, compensation=compensation)

    filename_split = filename.split(';')

    usable_files = []

    if not process_all:
        for fn in filename_split:
            fn = fn.strip()
            if is_valid_url(fn):
                dm, ltype, lid = deemix.parseLink(fn)
                if ltype and lid:
                    usable_files += download_deezer(fn, arl, track_format)
                else:
                    print('link indiriliyor...',end=' ')
                    usable_files+=[download_link(fn)]
                    print('bitti')
            else:
                usable_files.append(os.path.join('tracks',fn))
    else:
        for fn in glob.glob('tracks/*'):
            usable_files.append(fn)
    for filename in usable_files:
        suffix_naming = suffix if use_custom_parameter else None
        diff_suffix_naming = suffix_invert if use_custom_parameter else None
        run_mdx(onnx, mdx_model, filename, diff=invert,suffix=suffix_naming,diff_suffix=diff_suffix_naming,denoise=denoise)

    if print_settings:
        print()
        print('[MDX-Net_Colab ayarları kullanıldı]')
        print(f'Kullanılan model: {onnx}')
        print(f'MD5 Modeli: {mdx.MDX.get_hash(onnx)}')
        print(f'Gürültü giderme kullanılıyor: {denoise}')
        print(f'Model parametreleri:')
        print(f'    -dim_f: {mdx_model.dim_f}')
        print(f'    -dim_t: {mdx_model.dim_t}')
        print(f'    -n_fft: {mdx_model.n_fft}')
        print(f'    -compensation: {mdx_model.compensation}')
        print()
        print('[Input file]')
        print('filename(s): ')
        for filename in usable_files:
            print(f'    -{filename}')

    del mdx_model

# Klavuz

Bu eğitim kılavuzu, bu Colab not defterinin özelliklerini kullanma adımlarında size yol gösterecektir.

## Drive bağla

Google Drive'ınızı bağlamak için aşağıdaki adımları izleyin:

1. Google Drive'ı bağlamak istiyorsanız "MountDrive" seçeneğinin yanındaki kutuyu işaretleyin.
2. Bağlanacak sürücü için farklı bir yol belirtmek istiyorsanız "mounting_path" öğesini değiştirin. **Not:** Bu yolu değiştirirken dikkatli olun çünkü düzgün yapılmazsa sorunlara neden olabilir.
3. Deponuzdaki tüm yerel değişiklikleri atmak ve dosyaları orijinal commit'teki sürümlerle değiştirmek istiyorsanız "Güncellemeye zorla ve yerel değişiklikleri göz ardı et" seçeneğinin yanındaki kutuyu işaretleyin.
4. Değişikliklerinizi atmadan otomatik olarak güncellemek istiyorsanız "Otomatik Güncelle "nin yanındaki kutuyu işaretleyin. Manuel olarak güncellemek istiyorsanız işareti kaldırın.

## Girdi Dosyaları

Şarkılarınızı yüklemek için aşağıdaki adımları izleyin:

1. Şarkılarınız için "parça dosya adını" belirtin. Noktalı virgül (;) ile ayırarak birden fazla bağlantı veya dosya sağlayabilirsiniz.
2. Şarkılarınızı "tracks" klasörüne yükleyin.

## ONNX Modeli

Kendi ONNX modeliniz varsa, aşağıdaki adımları izleyin:

1. Modelinizi "models" klasörüne yükleyin.
2. Modeliniz için "onnx" dosya adını belirtin.

## İşleme

Parçalarınızı işlemek için aşağıdaki adımları izleyin:

1. "tracks" klasörü içindeki tüm parçaları işlemek istiyorsanız, "hepsini uygula" seçeneğinin yanındaki kutuyu işaretleyin ve "Dosya adı" alanını yok sayın.
2. İstediğiniz ek ayarları belirtin:
- Giriş ve çıkış arasındaki farkı almak için " ters çevir " in yanındaki kutuyu işaretleyin (örneğin, Vokallerden Enstrümantal elde edin).
   - MDX gürültüsünden kurtulmak için "denoise" yanındaki kutuyu işaretleyin. Bu, giriş parçasını iki kez işler.
   - Özel model parametrelerini yalnızca yeni/resmi olmayan/özel modeller kullanıyorsanız belirtin. Bu özelliği etkinleştirmek için " özel_parametreler kullan" onay kutusunu kullanın.
   - Genellikle kök adı olan çıktı dosyası son ekini belirtin (örneğin, Vokaller). Normal işleme için son eki belirtmek üzere "suffix" alanını ve ters çevrilmiş işleme için "suffix_invert" alanını kullanın.

## Model Parametreleri

Varsa aşağıdaki özel model parametrelerini belirtin:

- "dim_f": dim_f` parametresi için değer.
- "dim_t": dim_t` parametresi için değer.
- "n_fft": n_fft` parametresi için değer.
- Modeliniz için kendi telafi değeriniz varsa "use_custom_compensation" yanındaki kutuyu işaretleyin. Telafi değerini "compensation" alanında belirtin.

## Ekstralar

Deezer parçaları ile çalışıyorsanız, parçalara doğrudan erişmek için ARL'nizi (Authentication Request Library) "arl" alanına yapıştırın.

Parça için istediğiniz kaliteyi/formatı seçerek "Parça formatını" belirleyin.

Çalışmada kullanılan ayarları yazdırmak için "print_settings" yanındaki kutuyu işaretleyin.

İşte bu kadar! Artık bu Colab not defterini kullanmaya hazırsınız. Tadını çıkarın!

## Detaylı kılavuz için bu <a href="https://docs.google.com/document/d/17fjNvJzj8ZGSer7c7OFe_CNfUKbAxEh_OBv94ZdRG5c">link</a>e tıklayın.
credits: (discord) deton24